In [1]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = '3'

In [2]:
# Importing the libraries:
import tensorflow as tf
from tensorflow import keras
import numpy as np
tf.__version__, keras.__version__, tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


('2.10.0', '2.10.0', True)

# Prepping the Images:

In [3]:
# Using a function to automate image loading and transformations:
path = "data/"

data = tf.keras.preprocessing.image_dataset_from_directory(
    path,
    label_mode = None, # To ensure that it doesn't throw me an error and retrieves all the images.
    color_mode = "rgb",
    batch_size = 32,
    image_size = (96, 96),
    shuffle = True,
    
)
data = data.unbatch()
data = data.batch(32, drop_remainder = True)
for img in data:
    img = tf.cast(img, tf.float32)/255.

Found 21551 files belonging to 1 classes.


In [4]:
# Creating some more variables:
coding_size = 96
optimizer = tf.keras.optimizers.Nadam(learning_rate = 0.0005)
loss = tf.keras.losses.binary_crossentropy
batch_size = 32

# Creating a GAN:

In [5]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Conv2D, BatchNormalization, LeakyReLU, ReLU, Reshape, Conv2DTranspose, Activation, Flatten

In [6]:
# Weights initializer
init = tf.keras.initializers.RandomNormal(stddev=0.02)

In [7]:
def build_generator(seed_size):
    """
    Builds the generator model
    
    Parameters:
        seed_size: size of the random vector fed into the generator
    
    Returns:
        model: keras model representing the generator
    """
    model = Sequential()
  
    # Block - 1
    model.add(Dense(6*6*1024,kernel_initializer=init,input_dim=seed_size))
    model.add(BatchNormalization())
    model.add(ReLU())
    model.add(Reshape((6,6,1024))) # Resulting shape = (6,6,1024) 

    # Block - 2
    model.add(Conv2DTranspose(512,kernel_size=5,strides=2,padding='same',use_bias=False,kernel_initializer=init))
    model.add(BatchNormalization())
    model.add(ReLU())  # Resulting shape = (12,12,512)

    # Block - 3
    model.add(Conv2DTranspose(256,kernel_size=5,strides=2,padding='same',use_bias=False,kernel_initializer=init))
    model.add(BatchNormalization())
    model.add(ReLU()) # Resulting shape = (24,24,256)

    # Block - 4
    model.add(Conv2DTranspose(128,kernel_size=3,strides=2,padding='same',use_bias=False,kernel_initializer=init))
    model.add(BatchNormalization())
    model.add(ReLU()) # Resulting shape = (48,48,128)

    # Block - 5
    model.add(Conv2DTranspose(3,kernel_size=3,strides=2,padding='same',use_bias=False,kernel_initializer=init))
    model.add(Activation('tanh')) # Resulting shape = (96,96,3)

    return model

In [8]:
def build_discriminator(image_length,image_channels):
    
    """
    Builds the generator model
    
    Parameters:
        image_length: length of a side of the square image
        image_channels: number of channels in the image
    
    Returns:
        model: keras model representing the discriminator
    """
    
    model = Sequential()
  
    # Block - 1
    model.add(Conv2D(64,kernel_size=3,strides=2,padding='same',use_bias=False,input_shape=(image_length,image_length,image_channels),kernel_initializer=init))
    model.add(LeakyReLU(alpha=0.2)) # Resulting shape = (48,48,64)

    # Block - 2
    model.add(Conv2D(128,kernel_size=3,strides=2,padding='same',use_bias=False,kernel_initializer=init))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2)) # Resulting shape = (24,24,128)

    # Block - 3
    model.add(Conv2D(256,kernel_size=5,strides=2,padding='same',use_bias=False,kernel_initializer=init))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2)) # Resulting shape = (12,12,256)

    # Block - 4
    model.add(Conv2D(512,kernel_size=5,strides=2,padding='same',use_bias=False,kernel_initializer=init))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2)) # Resulting shape = (6,6,512)

    # Block - 5
    model.add(Conv2D(1,kernel_size=4,strides=1,padding='valid',use_bias=False,kernel_initializer=init))
    model.add(Flatten())
    model.add(Activation('sigmoid'))

    return model

In [9]:
# Creating a gan:
generator = build_generator(coding_size)
discriminator = build_discriminator(96, 3)
gan = keras.models.Sequential([generator, discriminator])

/home/impureking/miniconda3/envs/TensorFlow/lib/python3.10/site-packages/keras/initializers/initializers_v2.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


## Creating Custom Loss Functions

In [10]:
def discriminator_loss(real_output, fake_output, loss_fn = loss):
    real_loss = loss_fn(tf.ones_like(real_output), real_output)
    fake_loss = loss_fn(tf.zeros_like(fake_output), fake_output)
    total = real_loss + fake_loss
    return total

In [11]:
# Creating an generator loss:
def generator_loss(fake_output, loss_fn = loss):
    return loss_fn(tf.ones_like(fake_output), fake_output)

# Training the GAN:

In [12]:
# Creating a image plotter:
def plot(x_train, image_no = 1, no = 1):
    for i in tf.range(image_no):
        img = x_train[i]
        img = tf.keras.preprocessing.image.array_to_img(img)
        img.show()
        img.save("Progress/image{}.jpg".format(no))


In [13]:
# Creating an custom callback:
def Checkpoint(model):
    model.save("Checkpoints/GAN")

In [14]:
# General Step:
def gtrain_step(model, x, y, loss_fn = generator_loss, optimizer = optimizer):
    with tf.GradientTape() as tape:
        y_pred = model(x)
        loss = loss_fn(y_pred)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

In [15]:
def dtrain_step(model, x, y, loss_fn = discriminator_loss, optimizer = optimizer):
    with tf.GradientTape() as tape:
        y_pred = model(x)
        loss = loss_fn(fake_output = y_pred, real_output = y)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

In [16]:
#  Discriminator step:
@tf.function
def train_d(model, noise, x_batch, batch_size = batch_size):
    generator, discriminator = model.layers
    predictions = generator(noise)
    inputs = tf.concat([predictions, x_batch], axis = 0)
    y = tf.constant([[0.]] * batch_size + [[1.]] * batch_size)
    discriminator.trainable = True
    dtrain_step(discriminator, inputs, y)

In [17]:
# Generator step:
@tf.function
def train_g(model, noise, batch_size = batch_size):
    discriminator = model.layers[1]
    discriminator.trainable = False
    y = tf.constant([[1.]] * batch_size)
    gtrain_step(model, noise, y)

In [18]:
# Predicting values:
def predict(model, epoch, batch_size = batch_size, coding_size = coding_size):
    noise = tf.random.normal([batch_size, coding_size])
    generator = model.layers[0]
    predictions = generator(noise)
    plot(predictions, no = epoch)

In [19]:
# Training Loop:
epochs = 20
for epoch in range(epochs):
    print(f"Epochs {epoch + 1} started.")
    for x_batch in data:
        # Training the Discriminator:
        noise = tf.random.normal([batch_size, coding_size])
        train_d(gan, noise, x_batch)
        # Training the Generator:
        noise = tf.random.normal([batch_size, coding_size])
        train_g(gan, noise)
    predict(gan, epoch)

Epochs 1 started.
Epochs 2 started.
Epochs 3 started.
